In [1]:
from nfl_data_py import import_players, import_weekly_data

# Fetch player stats data
player_stats = import_weekly_data([2022])

# Fetch player metadata
players = import_players()

Downcasting floats.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Combine player stats and player metadata into a single DataFrame
df = pd.merge(player_stats, players, left_on='player_id', right_on='gsis_id', how='left')

# Sort the dataframe by player_id and week
df = df.sort_values(['player_id', 'week'])

# Create the target variable (next week's passing yards)
df['next_week_passing_yards'] = df.groupby('player_id')['passing_yards'].shift(-1)

# Remove the last week for each player (as we don't have next week's data for it)
df = df.groupby('player_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)

# Encode categorical features
label_encoder = LabelEncoder()
df['position_x'] = label_encoder.fit_transform(df['position_x'])
df['recent_team'] = label_encoder.fit_transform(df['recent_team'])

# Scale numeric features
scaler = StandardScaler()
df[['passing_yards', 'rushing_yards', 'receiving_yards']] = scaler.fit_transform(df[['passing_yards', 'rushing_yards', 'receiving_yards']])

# Remove rows with NaN values
df = df.dropna(subset=['next_week_passing_yards', 'passing_yards', 'rushing_yards', 'receiving_yards'])

In [3]:
X = df[['position_x', 'recent_team', 'passing_yards', 'rushing_yards', 'receiving_yards']]
y = df['next_week_passing_yards']

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Define the model architecture
model = Sequential()
model.add(Input(shape=(5,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

2024-08-21 16:48:38.775917: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:48:38.779111: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 16:48:38.786870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 16:48:38.800980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 16:48:38.805272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 16:48:38.816146: I tensorflow/core/platform/cpu_feature_gu

Epoch 1/100


I0000 00:00:1724280521.409582   45986 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-21 16:48:41.409977: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6451.8169 - val_loss: 4050.7029
Epoch 2/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5166.2427 - val_loss: 3026.7263
Epoch 3/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3388.3562 - val_loss: 1561.0391
Epoch 4/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1458.7260 - val_loss: 1310.9728
Epoch 5/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 924.9680 - val_loss: 1328.4873
Epoch 6/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1062.7841 - val_loss: 1299.3875
Epoch 7/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 929.3483 - val_loss: 1286.7583
Epoch 8/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1005.2951 - val_loss: 1280.9482
Epoch 9/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 962.0237 - val_loss: 1249.1046
Epoch 10/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 994.8852 - val_loss: 1258.3815
Epoch 11/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1007.7267 - val_loss: 1

In [5]:
import numpy as np

# Select a player (in this case, the first one in the DataFrame)
player_name = "Patrick Mahomes"  # Replace with the name of the player you're interested in
player_stats = df[df['player_display_name'] == player_name].iloc[0]

# Print player information
print(f"Player: {player_stats['player_name']} (ID: {player_stats['player_id']})")
print(f"Position: {player_stats['position_x']}")
print(f"Team: {player_stats['recent_team']}")
print(f"Current week stats:")
print(f"  Passing yards: {player_stats['passing_yards']}")
print(f"  Rushing yards: {player_stats['rushing_yards']}")
print(f"  Receiving yards: {player_stats['receiving_yards']}")

input_data = np.array([[
    player_stats['position_x'],
    player_stats['recent_team'],
    player_stats['passing_yards'],
    player_stats['rushing_yards'],
    player_stats['receiving_yards']
]])

next_week_passing_yards = None

try:
    prediction = model.predict(input_data)
    
    if prediction.shape == (1, 1):
        next_week_passing_yards = prediction[0][0]
        print(f"\nPredicted next week's passing yards: {next_week_passing_yards}")
    else:
        print(f"\nUnexpected prediction shape: {prediction.shape}")
        print(f"Prediction content: {prediction}")
        
except Exception as e:
    print(f"\nAn error occurred during prediction: {e}")

if next_week_passing_yards is None:
    print("\nFailed to generate a valid prediction.")

Player: P.Mahomes (ID: 00-0033873)
Position: 4
Team: 15
Current week stats:
  Passing yards: 4.503370548490142
  Rushing yards: -0.28853030402544877
  Receiving yards: -0.8017950927505366
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

Predicted next week's passing yards: 266.6775207519531
